# a9a dataset

In [6]:
import os
import sys
sys.path.append("../")
sys.path.insert(0, '../src')
from scipy.io import savemat, loadmat
from IPG.src.lossfunction import LogisticLoss
from IPG.src.regularizer_nocorrection import NatOG
from IPG.src.solver_nocorrection import IpgSolver
import IPG.src.utils as utils
from IPG.src.params import *
import yaml
import numpy as np

In [7]:
def main(lam_shrink = 0.1, datasetName = 'a9a', dbDir = '../../GroupFaRSA/db'):
    loss = 'logit'
    fileType = fileTypeDict[datasetName]
    print("Working on: {}...".format(datasetName))

    X, y = utils.set_up_xy(datasetName, fileType, dbDir)
    if loss == 'logit':
        f = LogisticLoss(X, y, datasetName)

    p = X.shape[1]
    grp_size = 10
    overlap_ratio = 0.2
    grp_size = min(p // 2, grp_size)
    generator = utils.GenOverlapGroup(
        p, grp_size=grp_size, overlap_ratio=overlap_ratio)
    groups, starts, ends = generator.get_group()

    lammax_path = f'{dbDir}/lammax-{datasetName}-{grp_size}-{overlap_ratio}.mat'
    if os.path.exists(lammax_path):
        lammax = loadmat(lammax_path)["lammax"][0][0]
        print(f"loading lammax from: {lammax_path}")
    else:
        lammax = utils.lam_max(X, y, starts, ends, loss)
        savemat(lammax_path, {"lammax": lammax})
        print(f"save lammax to: {lammax_path}")

    r = NatOG(penalty=lammax * lam_shrink, groups=groups, weights=None)


    alpha_init = 1.0


    print("Inexact subprobsolve: schimdt")
    with open('../IPG/src/config.yaml', "r") as stream:
        config = yaml.load(stream, Loader=yaml.SafeLoader)
    config['mainsolver']['exact_pg_computation'] = False
    config['mainsolver']['inexact_pg_computation'] = 'schimdt'
    config['inexactpg']['schimdt']['c'] = 1e4
    solver = IpgSolver(f, r, config)
    info_schimdt = solver.solve(alpha_init=alpha_init, save_ckpt=False)

    print("Inexact subprobsolve: lee")
    with open('../IPG/src/config.yaml', "r") as stream:
        config = yaml.load(stream, Loader=yaml.SafeLoader)
    config['mainsolver']['exact_pg_computation'] = False
    config['mainsolver']['inexact_pg_computation'] = 'lee'
    solver = IpgSolver(f, r, config)
    info_lee = solver.solve(alpha_init=alpha_init,  save_ckpt=False)


    with open('../IPG/src/config.yaml', "r") as stream:
        config = yaml.load(stream, Loader=yaml.SafeLoader)
    config['mainsolver']['exact_pg_computation'] = False
    config['mainsolver']['inexact_pg_computation'] = 'yd'
    solver = IpgSolver(f, r, config)
    info_yd = solver.solve(alpha_init=alpha_init,  save_ckpt=False)
    return info_schimdt, info_lee, info_yd

In [8]:
info_schimdt, info_lee, info_yd = main(lam_shrink = 0.1, datasetName = 'a9a', dbDir = '../../GroupFaRSA/db')

Working on: a9a...
loading lammax from: ../../GroupFaRSA/db/lammax-a9a-10-0.2.mat
Inexact subprobsolve: schimdt


ModuleNotFoundError: No module named 'src'

In [9]:
sys.path

['../src',
 '/home/yud319/InexactPG/AnalyzeExperiments',
 '',
 '/home/yud319/anaconda3/lib/python3.7/site-packages',
 '/home/yud319/anaconda3/lib/python37.zip',
 '/home/yud319/anaconda3/lib/python3.7',
 '/home/yud319/anaconda3/lib/python3.7/lib-dynload',
 '/home/yud319/.local/lib/python3.7/site-packages',
 '/home/yud319/anaconda3/lib/python3.7/site-packages/chardet-3.0.4-py3.7.egg',
 '/home/yud319/anaconda3/lib/python3.7/site-packages/docloud-1.0.375-py3.7.egg',
 '/home/yud319/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/home/yud319/.ipython',
 '../',
 '../']

In [1]:
import numpy as np

In [3]:
a = 299
b = 278

In [4]:
a + b

577